# Backtest Baselines Calculation Script

## Imports


In [1]:
import pandas as pd

## Calculate backtest baselines

### Get all companies data

In [2]:
all_companies = pd.read_csv('../STORAGE/COMPANIES.csv')
all_tickers = list(all_companies['ticker'])


### Initialize Dictionary to store Backtests baseline data

In [3]:
backtests_baselines_data = {}

keys = ['ticker','strategy_type_id','backtest_id','2017-2020']+[f'{2017 + yr}' for yr in range(4)] \
+[f'{2017 + yr}_Q{qtr}' for qtr in range(1, 5) for yr in range(4)] \
+[f'{2017 + yr}_M{mth}' for mth in range(1, 13) for yr in range(4)]

for key in keys:
	backtests_baselines_data[key] = []


### Function to calculate baseline percents

In [4]:
def calc_baseline_percent(ticker_data, start_index, end_index):
	return round(((ticker_data['close'][end_index] - ticker_data['close'][start_index]) / ticker_data['close'][start_index]) * 100, 2)

### Function to get all trades for a backtests

In [5]:
def get_all_trades_of_a_backtest(ticker, backtest_id):

	# Get backtest trades
	backtest_trade_df = pd.read_csv(f'../STORAGE/BACKTEST_TRADES/{ticker}.csv')
	backtest_trade_df = backtest_trade_df[backtest_trade_df['back_test_report_id'] == backtest_id]

	return list(backtest_trade_df['trade_id'])


### Function to get return % of each trade id

In [6]:
def get_return_percents(ticker, trade_ids):

	# Initialize list to return
	trade_return_percents = []

	# Get data of trades
	trades_df = pd.read_csv(f'../STORAGE/TRADES/{ticker}.csv')
	trades_df = trades_df[trades_df['id'].isin(trade_ids)]
	trades_df = trades_df.sort_values('id')

	return list(trades_df['return_percent'])


### Function to calculate total returns %

In [7]:
def calc_total_return_percent(return_percents):
	initial = 100

	for return_percent in return_percents:
		initial *= (1 + (return_percent/100))
	
	return round(initial - 100, 2)


### Function to get trade entry dates

In [8]:
def get_time_stamps(ticker, trade_ids):

	# Get trades
	trades_df = pd.read_csv(f'../STORAGE/TRADES/{ticker}.csv')
	trades_df = trades_df[trades_df['id'].isin(trade_ids)]
	trades_df = trades_df.sort_values('id')

	entry_order_ids = list(trades_df['entry_order_id'])

	# Get Orders
	orders_df = pd.read_csv(f'../STORAGE/ORDERS/{ticker}.csv')
	orders_df = orders_df[orders_df['id'].isin(entry_order_ids)]
	orders_df = orders_df.sort_values('id')

	ticker_data_ids = list(orders_df['ticker_data_id'])

	# Get timestamps
	ticker_data_df = pd.read_csv(f'../STORAGE/TICKER_DATA/{ticker}.csv')
	ticker_data_df = ticker_data_df[ticker_data_df['id'].isin(ticker_data_ids)]
	ticker_data_df = ticker_data_df.sort_values('id')

	return list(ticker_data_df['time_stamp'])


### Calculate baselines

In [9]:
# For each company
for ticker in all_tickers:
	
	# Get company backtests Data
	company_backtests = pd.read_csv(f"../STORAGE/BACKTEST_REPORTS/{ticker}.csv", parse_dates=True)
	
	# For each backtest in company
	for backtest in range(company_backtests.index[0], company_backtests.index[-1]+1):
		
		# Add company ticker
		backtests_baselines_data['ticker'].append(ticker)
		# Add strategy id
		backtests_baselines_data['strategy_type_id'].append(company_backtests['strategy_type_id'][backtest])
		# Add backtest ids
		backtests_baselines_data['backtest_id'].append(company_backtests['id'][backtest])
        
		# Get all trades in a backtest
		backtest_trades = get_all_trades_of_a_backtest(ticker, company_backtests['id'][backtest])
		
		# Calculations
		if backtest_trades:

			trades_df = pd.DataFrame()

			# Get return % for each trade
			return_percents = get_return_percents(ticker, backtest_trades)
			trades_df['return_percents'] = return_percents

			# Get time stamps for each trade
			trade_time_stamps_df = get_time_stamps(ticker, backtest_trades)
			trades_df['time_stamps'] = trade_time_stamps_df

			# Calculate total returns percent
			total_returns_percent = calc_total_return_percent(return_percents)

			yearly_returns = {}
			# Calculate yearly returns
			for year in range(2017, 2021):
				mask = (trades_df['time_stamps'] >= f'{year}-01-01') & (trades_df['time_stamps'] < f'{year+1}-01-01')
				year_trades_df = trades_df.loc[mask]
				yearly_returns[f'{year}'] = calc_total_return_percent(list(year_trades_df['return_percents']))
			
			quaterly_returns = {}
			# Calculate quaterly returns
			for year in range(2017, 2021):
				quater_count = 0
				for qtr in [(f'{year}-01-01', f'{year}-04-01'), 
							(f'{year}-04-01', f'{year}-07-01'), 
							(f'{year}-07-01', f'{year}-10-01'), 
							(f'{year}-10-01', f'{year+1}-01-01')]:
					quater_count += 1
					mask = (trades_df['time_stamps'] >= qtr[0]) & (trades_df['time_stamps'] < qtr[1])
					quater_trades_df = trades_df.loc[mask]
					quaterly_returns[f'{year}_Q{quater_count}'] = calc_total_return_percent(list(quater_trades_df['return_percents']))
			
			monthly_returns = {}
			# Calculate monthly returns
			for year in range(2017, 2021):

				for month in range(1, 13):

					start_date = f'{year}-{month}-01'
					end_date = f'{year}-{month+1}-01'

					if month == 12:
						end_date = f'{year+1}-01-01'
					elif month == 9:
						start_date = f'{year}-0{month}-01'
						end_date = f'{year}-{month+1}-01'
					elif month < 10:
						start_date = f'{year}-0{month}-01'
						end_date = f'{year}-0{month+1}-01'
					
					mask = (trades_df['time_stamps'] >= start_date) & (trades_df['time_stamps'] < end_date)
					monthly_trades_df = trades_df.loc[mask]
					monthly_returns[f'{year}_M{month}'] = calc_total_return_percent(list(monthly_trades_df['return_percents']))
	
		else:
			total_returns_percent = 0

			yearly_returns = {}
			for year in range(2017, 2021):
				yearly_returns[f'{year}'] = 0.0
			
			quaterly_returns = {}
			for year in range(2017, 2021):
				for qtr in range(1, 5):
					quaterly_returns[f'{year}_Q{qtr}'] = 0.0
			
			monthly_returns = {}
			for year in range(2017, 2021):
				for month in range(1, 13):
					monthly_returns[f'{year}_M{month}'] = 0.0

		# Add total return percent of backtests
		backtests_baselines_data['2017-2020'].append(total_returns_percent)

		# Add yearly returns
		for year in yearly_returns:
			backtests_baselines_data[year].append(yearly_returns[year])

		# Add quaterly returns
		for qtr in quaterly_returns:
			backtests_baselines_data[qtr].append(quaterly_returns[qtr])
		
		# Add monthy returns
		for month in monthly_returns:
			backtests_baselines_data[month].append(monthly_returns[month])

		# Comment to run for all backtests for a company
		# break

	# Comment to run for all 50 companies
	# break


## Convert data dictionary into a dataframe

In [15]:
backtests_baseline_df = pd.DataFrame.from_dict(backtests_baselines_data)
backtests_baseline_df.tail()




,ticker,strategy_type_id,backtest_id,2017-2020,2017,2018,2019,2020,2017_Q1,2018_Q1,...,2019_M10,2020_M10,2017_M11,2018_M11,2019_M11,2020_M11,2017_M12,2018_M12,2019_M12,2020_M12
1995,WIPRO.NS,2,1996,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1996,WIPRO.NS,2,1997,-45.92,-13.42,4.13,-19.26,-25.71,-6.69,10.95,...,-3.65,-4.37,2.23,3.50,-5.79,-1.52,-3.72,6.85,0.00,0.00
1997,WIPRO.NS,2,1998,-90.52,-49.37,55.45,-45.51,-77.89,-2.44,18.71,...,-7.06,15.55,-2.50,20.60,1.86,-3.87,-19.59,12.72,-4.59,-2.75
1998,WIPRO.NS,2,1999,-85.95,-45.17,60.97,-40.45,-73.26,-6.49,19.90,...,-3.91,5.01,4.19,16.25,-4.81,-12.47,-10.85,16.03,3.51,0.00
1999,WIPRO.NS,2,2000,-62.28,2.27,-0.67,-2.34,-61.98,-7.20,2.02,...,4.38,30.01,1.93,5.10,15.41,-8.16,3.46,0.00,3.40,0.00


In [12]:
print(backtests_baseline_df)

             ticker  strategy_type_id  backtest_id  2017-2020   2017   2018  \
0     ADANIPORTS.NS                 1            1      14.38  16.39 -11.32   
1     ADANIPORTS.NS                 1            2       0.00   0.00   0.00   
2     ADANIPORTS.NS                 1            3     -16.22  14.57 -10.43   
3     ADANIPORTS.NS                 1            4       0.00   0.00   0.00   
4     ADANIPORTS.NS                 1            5      32.68   8.77  -9.18   
...             ...               ...          ...        ...    ...    ...   
1995       WIPRO.NS                 2         1996       0.00   0.00   0.00   
1996       WIPRO.NS                 2         1997     -45.92 -13.42   4.13   
1997       WIPRO.NS                 2         1998     -90.52 -49.37  55.45   
1998       WIPRO.NS                 2         1999     -85.95 -45.17  60.97   
1999       WIPRO.NS                 2         2000     -62.28   2.27  -0.67   

        2019   2020  2017_Q1  2018_Q1  ...  2019_M1

## Store data in a csv

In [11]:
backtests_baseline_df.to_csv('../baselines/BACKTESTS_BASELINES.csv', index=False)